In [1]:
%%writefile Keras_Model_Mae.py

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, AveragePooling2D, GlobalAveragePooling2D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet import ResNet50

def load_train(path):
    lables = pd.read_csv(path + 'labels.csv')
    train_datagen = ImageDataGenerator(rescale=1./255, 
                                       vertical_flip=True, 
                                       horizontal_flip=True, 
                                       #rotation_range=30, 
                                       #width_shift_range=0.2,
                                       #height_shift_range=0.2
                                       validation_split=0.25
                                      )
    train_datagen_flow = train_datagen.flow_from_dataframe(
        dataframe = lables, 
        directory = path + 'final_files/', 
        x_col = 'file_name', 
        y_col = 'real_age',
        target_size = (224,224),
        class_mode = 'raw', 
        batch_size = 32,
        subset = 'training',
        seed = 12345)

    return train_datagen_flow



def load_test(path):
    lables = pd.read_csv(path + 'labels.csv')
    test_datagen = ImageDataGenerator(rescale=1./255, 
                                      validation_split=0.25
                                     )
    test_datagen_flow = test_datagen.flow_from_dataframe(
        dataframe = lables, 
        directory = path + 'final_files/', 
        x_col = 'file_name', 
        y_col = 'real_age', 
        class_mode = 'raw',
        target_size = (224,224),
        batch_size = 32,
        subset = 'validation',
        seed = 12345)

    return test_datagen_flow
    
def create_model(input_shape):
    backbone = ResNet50(input_shape=input_shape,
                        weights='imagenet', 
                        include_top=False)
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1))
    model.compile(optimizer=Adam(lr=0.0001), loss='mean_squared_error',metrics=['mae'])

    return model

def train_model(model, train_data, test_data, batch_size=None, epochs=5,steps_per_epoch=None, validation_steps=None):
    
    if steps_per_epoch is None:
                 steps_per_epoch = len(train_data)
    if validation_steps is None:
                  validation_steps = len(test_data)
            
    model.fit(train_data,
          validation_data=test_data,
          batch_size=batch_size, epochs=epochs,
          steps_per_epoch=steps_per_epoch,
          validation_steps=validation_steps,
          verbose=2, shuffle=True)
    
    return model

Writing Keras_Model_Mae.py
